In [1]:
import matplotlib as mpl
import matplotlib.pyplot as plt
%matplotlib inline
import numpy as np
import sklearn
import pandas as pd
import os
import sys
import time
import tensorflow as tf

from tensorflow import keras

print(tf.__version__)
print(sys.version_info)
for module in mpl, np, pd, sklearn, tf, keras:
    print(module.__name__, module.__version__)

2.2.0
sys.version_info(major=3, minor=6, micro=9, releaselevel='final', serial=0)
matplotlib 3.2.1
numpy 1.18.5
pandas 1.0.4
sklearn 0.23.1
tensorflow 2.2.0
tensorflow.keras 2.3.0-tf


In [2]:
# https://storage.googleapis.com/download.tensorflow.org/data/shakespeare.txt
input_filepath = "./shakespeare.txt"
text = open(input_filepath, 'r').read()

print(len(text))
print(text[0:100])

1115394
First Citizen:
Before we proceed any further, hear me speak.

All:
Speak, speak.

First Citizen:
You


In [3]:
# 1. generate vocab
# 2. build mapping char->id
# 3. data -> id_data
# 4. abcd -> bcd<eos>

vocab = sorted(set(text))
print(len(vocab))
print(vocab)

65
['\n', ' ', '!', '$', '&', "'", ',', '-', '.', '3', ':', ';', '?', 'A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'J', 'K', 'L', 'M', 'N', 'O', 'P', 'Q', 'R', 'S', 'T', 'U', 'V', 'W', 'X', 'Y', 'Z', 'a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'm', 'n', 'o', 'p', 'q', 'r', 's', 't', 'u', 'v', 'w', 'x', 'y', 'z']


In [4]:
char2idx = {char:idx for idx, char in enumerate(vocab)}
print(char2idx)

{'\n': 0, ' ': 1, '!': 2, '$': 3, '&': 4, "'": 5, ',': 6, '-': 7, '.': 8, '3': 9, ':': 10, ';': 11, '?': 12, 'A': 13, 'B': 14, 'C': 15, 'D': 16, 'E': 17, 'F': 18, 'G': 19, 'H': 20, 'I': 21, 'J': 22, 'K': 23, 'L': 24, 'M': 25, 'N': 26, 'O': 27, 'P': 28, 'Q': 29, 'R': 30, 'S': 31, 'T': 32, 'U': 33, 'V': 34, 'W': 35, 'X': 36, 'Y': 37, 'Z': 38, 'a': 39, 'b': 40, 'c': 41, 'd': 42, 'e': 43, 'f': 44, 'g': 45, 'h': 46, 'i': 47, 'j': 48, 'k': 49, 'l': 50, 'm': 51, 'n': 52, 'o': 53, 'p': 54, 'q': 55, 'r': 56, 's': 57, 't': 58, 'u': 59, 'v': 60, 'w': 61, 'x': 62, 'y': 63, 'z': 64}


In [5]:
idx2char = np.array(vocab)
print(idx2char)

['\n' ' ' '!' '$' '&' "'" ',' '-' '.' '3' ':' ';' '?' 'A' 'B' 'C' 'D' 'E'
 'F' 'G' 'H' 'I' 'J' 'K' 'L' 'M' 'N' 'O' 'P' 'Q' 'R' 'S' 'T' 'U' 'V' 'W'
 'X' 'Y' 'Z' 'a' 'b' 'c' 'd' 'e' 'f' 'g' 'h' 'i' 'j' 'k' 'l' 'm' 'n' 'o'
 'p' 'q' 'r' 's' 't' 'u' 'v' 'w' 'x' 'y' 'z']


In [6]:
text_as_int = np.array([char2idx[c] for c in text])
print(text_as_int[0:10])
print(text[0:10])

[18 47 56 57 58  1 15 47 58 47]
First Citi


In [7]:
def split_input_target(id_text):
    """
    abcde -> abcd, bcde
    """
    return id_text[0:-1], id_text[1:]

char_dataset = tf.data.Dataset.from_tensor_slices(text_as_int)
seq_length = 100
seq_dataset = char_dataset.batch(seq_length + 1,
                                 drop_remainder = True)
for ch_id in char_dataset.take(2):
    print(ch_id, idx2char[ch_id.numpy()])

for seq_id in seq_dataset.take(2):
    print(seq_id)
    print(repr(''.join(idx2char[seq_id.numpy()])))

tf.Tensor(18, shape=(), dtype=int64) F
tf.Tensor(47, shape=(), dtype=int64) i
tf.Tensor(
[18 47 56 57 58  1 15 47 58 47 64 43 52 10  0 14 43 44 53 56 43  1 61 43
  1 54 56 53 41 43 43 42  1 39 52 63  1 44 59 56 58 46 43 56  6  1 46 43
 39 56  1 51 43  1 57 54 43 39 49  8  0  0 13 50 50 10  0 31 54 43 39 49
  6  1 57 54 43 39 49  8  0  0 18 47 56 57 58  1 15 47 58 47 64 43 52 10
  0 37 53 59  1], shape=(101,), dtype=int64)
'First Citizen:\nBefore we proceed any further, hear me speak.\n\nAll:\nSpeak, speak.\n\nFirst Citizen:\nYou '
tf.Tensor(
[39 56 43  1 39 50 50  1 56 43 57 53 50 60 43 42  1 56 39 58 46 43 56  1
 58 53  1 42 47 43  1 58 46 39 52  1 58 53  1 44 39 51 47 57 46 12  0  0
 13 50 50 10  0 30 43 57 53 50 60 43 42  8  1 56 43 57 53 50 60 43 42  8
  0  0 18 47 56 57 58  1 15 47 58 47 64 43 52 10  0 18 47 56 57 58  6  1
 63 53 59  1 49], shape=(101,), dtype=int64)
'are all resolved rather to die than to famish?\n\nAll:\nResolved. resolved.\n\nFirst Citizen:\nFirst, you k'


In [8]:
seq_dataset = seq_dataset.map(split_input_target)

for item_input, item_output in seq_dataset.take(2):
    print(item_input.numpy())
    print(item_output.numpy())

[18 47 56 57 58  1 15 47 58 47 64 43 52 10  0 14 43 44 53 56 43  1 61 43
  1 54 56 53 41 43 43 42  1 39 52 63  1 44 59 56 58 46 43 56  6  1 46 43
 39 56  1 51 43  1 57 54 43 39 49  8  0  0 13 50 50 10  0 31 54 43 39 49
  6  1 57 54 43 39 49  8  0  0 18 47 56 57 58  1 15 47 58 47 64 43 52 10
  0 37 53 59]
[47 56 57 58  1 15 47 58 47 64 43 52 10  0 14 43 44 53 56 43  1 61 43  1
 54 56 53 41 43 43 42  1 39 52 63  1 44 59 56 58 46 43 56  6  1 46 43 39
 56  1 51 43  1 57 54 43 39 49  8  0  0 13 50 50 10  0 31 54 43 39 49  6
  1 57 54 43 39 49  8  0  0 18 47 56 57 58  1 15 47 58 47 64 43 52 10  0
 37 53 59  1]
[39 56 43  1 39 50 50  1 56 43 57 53 50 60 43 42  1 56 39 58 46 43 56  1
 58 53  1 42 47 43  1 58 46 39 52  1 58 53  1 44 39 51 47 57 46 12  0  0
 13 50 50 10  0 30 43 57 53 50 60 43 42  8  1 56 43 57 53 50 60 43 42  8
  0  0 18 47 56 57 58  1 15 47 58 47 64 43 52 10  0 18 47 56 57 58  6  1
 63 53 59  1]
[56 43  1 39 50 50  1 56 43 57 53 50 60 43 42  1 56 39 58 46 43 56  1 58
 53  1 42

In [9]:
batch_size = 64
buffer_size = 10000

seq_dataset = seq_dataset.shuffle(buffer_size).batch(
    batch_size, drop_remainder=True)


In [10]:
vocab_size = len(vocab)
embedding_dim = 256
rnn_units = 1024

def build_model(vocab_size, embedding_dim, rnn_units, batch_size):
    model = keras.models.Sequential([
        keras.layers.Embedding(vocab_size, embedding_dim,
                               batch_input_shape = [batch_size, None]),
        keras.layers.LSTM(units = rnn_units,
                          #如果为True，则将批次中索引i的每个样本的最后状态用作下一个批次中索引i的样本的初始状态。
                          stateful = True,
                          # 权重矩阵的 初始化程序，用于循环状态的线性转换
                          recurrent_initializer = 'glorot_uniform',
                          return_sequences = True),
        keras.layers.Dense(vocab_size),
    ])
    return model

model = build_model(
    vocab_size = vocab_size,
    embedding_dim = embedding_dim,
    rnn_units = rnn_units,
    batch_size = batch_size)

model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (64, None, 256)           16640     
_________________________________________________________________
lstm (LSTM)                  (64, None, 1024)          5246976   
_________________________________________________________________
dense (Dense)                (64, None, 65)            66625     
Total params: 5,330,241
Trainable params: 5,330,241
Non-trainable params: 0
_________________________________________________________________


In [11]:
for input_example_batch, target_example_batch in seq_dataset.take(1):
    example_batch_predictions = model(input_example_batch)
    print(example_batch_predictions.shape)

(64, 100, 65)


In [12]:
# random sampling.
# greedy, random.
sample_indices = tf.random.categorical(
    logits = example_batch_predictions[0], num_samples = 1)
print(sample_indices)
# (100, 65) -> (100, 1)
sample_indices = tf.squeeze(sample_indices, axis = -1)
print(sample_indices)

tf.Tensor(
[[64]
 [54]
 [ 2]
 [ 3]
 [39]
 [59]
 [36]
 [37]
 [23]
 [50]
 [20]
 [ 4]
 [19]
 [ 8]
 [58]
 [15]
 [37]
 [24]
 [46]
 [ 9]
 [43]
 [59]
 [17]
 [10]
 [54]
 [53]
 [10]
 [29]
 [ 8]
 [27]
 [60]
 [ 7]
 [14]
 [35]
 [ 8]
 [40]
 [33]
 [45]
 [45]
 [34]
 [48]
 [12]
 [61]
 [55]
 [10]
 [64]
 [54]
 [60]
 [49]
 [23]
 [48]
 [43]
 [18]
 [ 4]
 [59]
 [37]
 [45]
 [58]
 [25]
 [ 1]
 [46]
 [49]
 [ 3]
 [38]
 [33]
 [42]
 [47]
 [ 4]
 [48]
 [36]
 [54]
 [59]
 [47]
 [38]
 [37]
 [17]
 [ 2]
 [27]
 [19]
 [50]
 [63]
 [37]
 [37]
 [36]
 [52]
 [48]
 [48]
 [ 8]
 [30]
 [ 0]
 [48]
 [13]
 [37]
 [29]
 [37]
 [35]
 [36]
 [28]
 [28]
 [47]], shape=(100, 1), dtype=int64)
tf.Tensor(
[64 54  2  3 39 59 36 37 23 50 20  4 19  8 58 15 37 24 46  9 43 59 17 10
 54 53 10 29  8 27 60  7 14 35  8 40 33 45 45 34 48 12 61 55 10 64 54 60
 49 23 48 43 18  4 59 37 45 58 25  1 46 49  3 38 33 42 47  4 48 36 54 59
 47 38 37 17  2 27 19 50 63 37 37 36 52 48 48  8 30  0 48 13 37 29 37 35
 36 28 28 47], shape=(100,), dtype=int64)


In [13]:
print("Input: ", repr("".join(idx2char[input_example_batch[0]])))
print()
print("Output: ", repr("".join(idx2char[target_example_batch[0]])))
print()
print("Predictions: ", repr("".join(idx2char[sample_indices])))

Input:  'to this woman?\n\nANGELO:\nI was, my lord.\n\nDUKE VINCENTIO:\nGo take her hence, and marry her instantly.'

Output:  'o this woman?\n\nANGELO:\nI was, my lord.\n\nDUKE VINCENTIO:\nGo take her hence, and marry her instantly.\n'

Predictions:  'zp!$auXYKlH&G.tCYLh3euE:po:Q.Ov-BW.bUggVj?wq:zpvkKjeF&uYgtM hk$ZUdi&jXpuiZYE!OGlyYYXnjj.R\njAYQYWXPPi'


In [14]:
def loss(labels, logits):
    return keras.losses.sparse_categorical_crossentropy(
        labels, logits, from_logits=True)

model.compile(optimizer = 'adam', loss = loss)
example_loss = loss(target_example_batch, example_batch_predictions)
print(example_loss.shape)
print(example_loss.numpy().mean())

(64, 100)
4.1746783


In [16]:
output_dir = "./text_generation_lstm3_checkpoints"
if not os.path.exists(output_dir):
    os.mkdir(output_dir)
checkpoint_prefix = os.path.join(output_dir, 'ckpt_{epoch}')
checkpoint_callback = keras.callbacks.ModelCheckpoint(
    filepath = checkpoint_prefix,
    save_weights_only = True)

epochs = 10
history = model.fit(seq_dataset, epochs = epochs,
                    callbacks = [checkpoint_callback])


Epoch 1/10
172/172 [==============================] - 598s 3s/step - loss: 1.7669
Epoch 2/10
172/172 [==============================] - 603s 4s/step - loss: 1.5752
Epoch 3/10
172/172 [==============================] - 579s 3s/step - loss: 1.4681
Epoch 4/10
172/172 [==============================] - 568s 3s/step - loss: 1.3994
Epoch 5/10
172/172 [==============================] - 573s 3s/step - loss: 1.3486
Epoch 6/10
172/172 [==============================] - 598s 3s/step - loss: 1.3072
Epoch 7/10
172/172 [==============================] - 599s 3s/step - loss: 1.2699
Epoch 8/10
172/172 [==============================] - 612s 4s/step - loss: 1.2332
Epoch 9/10
172/172 [==============================] - 611s 4s/step - loss: 1.1989
Epoch 10/10
172/172 [==============================] - 608s 4s/step - loss: 1.1635


In [17]:
tf.train.latest_checkpoint(output_dir)

'./text_generation_lstm3_checkpoints/ckpt_10'

In [18]:
model2 = build_model(vocab_size,
                     embedding_dim,
                     rnn_units,
                     batch_size = 1)
model2.load_weights(tf.train.latest_checkpoint(output_dir))
model2.build(tf.TensorShape([1, None]))
# start ch sequence A,
# A -> model -> b
# A.append(b) -> B
# B(Ab) -> model -> c
# B.append(c) -> C
# C(Abc) -> model -> ...
model2.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_1 (Embedding)      (1, None, 256)            16640     
_________________________________________________________________
lstm_1 (LSTM)                (1, None, 1024)           5246976   
_________________________________________________________________
dense_1 (Dense)              (1, None, 65)             66625     
Total params: 5,330,241
Trainable params: 5,330,241
Non-trainable params: 0
_________________________________________________________________


In [19]:
def generate_text(model, start_string, num_generate = 1000):
    input_eval = [char2idx[ch] for ch in start_string]
    input_eval = tf.expand_dims(input_eval, 0)
    
    text_generated = []
    model.reset_states()
#另外一个变动的地方是这里
    # temperature > 1, 概率平缓，更加random  可以设置为2
    # temperature < 1, 概率更陡峭  greedy   贪婪的，可以设为0.5试试
    temperature = 2
    
    for _ in range(num_generate):
        # 1. model inference -> predictions
        # 2. sample -> ch -> text_generated.
        # 3. update input_eval
        
        # predictions : [batch_size, input_eval_len, vocab_size]
        predictions = model(input_eval)
        # predictions: logits -> softmax -> prob  logits做了softmax之后变为概率
        # softmax: e^xi 
        # eg: 4,2 e^4/(e^4 + e^2) = 0.88, e^2 / (e^4 + e^2) = 0.12
        #如果都除以2，变为2,1，会发现概率相对于原来，没有那么陡峭了
        # eg: 2,1 e^2/(e^2 + e) = 0.73, e / (e^2 + e) = 0.27
        predictions = predictions / temperature
        # predictions : [input_eval_len, vocab_size]
        predictions = tf.squeeze(predictions, 0)
        # predicted_ids: [input_eval_len, 1]
        # a b c -> b c d
        predicted_id = tf.random.categorical(
            predictions, num_samples = 1)[-1, 0].numpy()
        text_generated.append(idx2char[predicted_id])
        # s, x -> rnn -> s', y
        input_eval = tf.expand_dims([predicted_id], 0)
    return start_string + ''.join(text_generated)

new_text = generate_text(model2, "All: ")
print(new_text)
#会看出效果比原来好了很多

All: s;
And thmugh such liney-batf: neven wis oemed; yed t:
'3:
Thhich! $rancet o'd? hanonighNAWIRSIO ONGBUM Mithou didstusee! fram ame, y'll kiss.
How? shall Compurre'?
O, peans, tawinom,
And cawns!

FERRWIO:
O viry,!
It mightnlewhepp'd? I thick till lose.
Mwhanthed$H
me DHHY!
I am plebeipaoe ame, wherein Lowd ordf, devise I; and
cundy LNCO:
MPABIO:
An; ince to
p; 'Tisloour of thy;
'wheak niscopteal fearfuls Pourd.
Bears, upono3'IG Oun.
Too;---titl?
Hat, cave Phwent? prepenion: England!'?
Zet night saftly, ownwinhark-mo't.
Romeo
Little ry or.
I sacriLT:
Ay; uny:
Increoce fridon, lie. Yield!

LAETUSOP-
Leardloman, of Rame.'Yony: configu foo,, I hjat.
Dossedurb yex: kernaSelRE camia.! Am so. Piritfale XIncvapour
'Tas ie heariem; up well didoPEVEY:
I' Tw.
AliXANS
Otword he berrylyies;
Breatships Mx$ensanying?
Not agged enot
Ca-ull humph by the probott anyour
poitty Posh,
Highwivermunomen.
Wolth arr dlvio: tow, we put cufs
inde aryermonce.

Fir, when d, I'll RMQEXESin'tbscupe? is yrum RCA